In [ ]:
%pip install setuptools
%pip install ydata_profiling
%pip install scikit-learn
%pip install xlrd
%pip install openpyxl
%pip install ipywidgets
%pip install statsmodels
%pip install catboost

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from ydata_profiling import ProfileReport
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import dendrogram, linkage
from statsmodels.tsa.stattools import adfuller
from catboost import CatBoostRegressor
import catboost
from catboost import * 

In [ ]:
train=pd.read_excel("train.xlsx")
test=pd.read_excel("test.xlsx")

In [ ]:
X_train = train.drop("out", axis = 1)
X_test = test.drop("out", axis = 1)
X = pd.concat([X_train, X_test], axis=0)
y_train = train["out"]
y_test = test["out"]

In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21408 entries, 0 to 21407
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       21408 non-null  int64  
 1   id               21408 non-null  int64  
 2   month_of_period  21408 non-null  int64  
 3   year             21408 non-null  int64  
 4   quarter          21408 non-null  int64  
 5   month            21408 non-null  int64  
 6   cargo_volume     21408 non-null  int64  
 7   cargo_sales      21408 non-null  int64  
 8   innactive        21408 non-null  int64  
 9   active           21408 non-null  int64  
 10  outflow          21408 non-null  float64
 11  cargo_price      21408 non-null  float64
dtypes: float64(2), int64(10)
memory usage: 2.0 MB


In [ ]:
cat_model = CatBoostRegressor(loss_function="MAPE", depth=16, iterations =  100, task_type="GPU", eval_metric="SMAPE")

cat_model.fit(
    X_train, y_train, use_best_model=False,
    eval_set=(X_test, y_test),
    verbose=True,  
    plot=True,
)
cat_model.save_model("cat_model")

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Default metric period is 5 because SMAPE is/are not implemented for GPU
Metric SMAPE is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	learn: 198.1343403	test: 198.3118035	best: 198.3118035 (0)	total: 55.4ms	remaining: 5.49s
1:	total: 64.7ms	remaining: 3.17s
2:	total: 72.8ms	remaining: 2.35s
3:	total: 84.5ms	remaining: 2.03s
4:	total: 161ms	remaining: 3.06s
5:	learn: 190.2707098	test: 191.2065359	best: 191.2065359 (5)	total: 219ms	remaining: 3.43s
6:	total: 391ms	remaining: 5.19s
7:	total: 411ms	remaining: 4.72s
8:	total: 579ms	remaining: 5.86s
9:	total: 749ms	remaining: 6.74s
10:	learn: 184.3397357	test: 185.8278423	best: 185.8278423 (10)	total: 922ms	remaining: 7.46s
11:	total: 1.07s	remaining: 7.88s
12:	total: 1.25s	remaining: 8.36s
13:	total: 1.42s	remaining: 8.73s
14:	total: 1.59s	remaining: 9.03s
15:	learn: 179.5271700	test: 181.4813577	best: 181.4813577 (15)	total: 1.77s	remaining: 9.28s
16:	total: 1.94s	remaining: 9.47s
17:	total: 2.11s	remaining: 9.62s
18:	total: 2.28s	remaining: 9.73s
19:	total: 2.45s	remaining: 9.82s
20:	learn: 175.5950494	test: 177.9215177	best: 177.9215177 (20)	total: 2.63s	remaining: 

In [ ]:
# cat_model.score()

In [ ]:
lr_model = LogisticRegression(max_iter = 100)  
lr_model.fit(X_train,y_train)
predictions = lr_model.predict(X_test, )
probabilities = lr_model.predict_proba(X_test)[:,1]
acc = accuracy_score(y_test, predictions)
precision = precision_score (y_test, predictions)
recall = recall_score (y_test, predictions)
print(acc, precision, recall) # вывод на экран метрик, характеризующих качество модели
#lr_model.save_model("lr_model")


In [ ]:
rf_model = RandomForestClassifier(n_estimators = 100)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test) 
acc_rf = accuracy_score(y_test, y_pred)
precision_rf = precision_score (y_test, y_pred)
recall_rf = recall_score (y_test, y_pred)
print(acc_rf, precision_rf, recall_rf)# вывод на экран метрик, характеризующих качество модели
#rf_model.save_model("lr_model")